# 训练

In [ ]:
#导入所需的包
import numpy as np
import pickle as pk
import os, sys
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score 
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential,Model
from keras.layers import Conv1D, MaxPooling1D,Input,Activation,Add
from keras.layers import Flatten,Dense,LSTM,Bidirectional,GlobalAveragePooling1D,Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


In [2]:
def get_session(gpu_fraction=0.1):
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
    return tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
tf.compat.v1.keras.backend.set_session(get_session())

In [3]:
fn = "data_intra.pk"
with open(fn, "rb") as fp:
    X_train = pk.load(fp)
    Y_train = pk.load(fp)
    X_valid = pk.load(fp)
    Y_valid = pk.load(fp)
    X_test = pk.load(fp)
    Y_test = pk.load(fp)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(65411, 201, 1)
(19244, 201, 1)
(22671, 201, 1)


In [4]:
def Conv_BN(input_name,num_filter,kernel_size):
#     x = Conv1D(18, 7)(input_name)
    x = Conv1D(num_filter,kernel_size,padding='same')(input_name)
    x1 = BatchNormalization()(x)
    return Activation('relu')(x1)

def Conv_Res(input_name,num_filter,kernel_size):
    x = Conv_BN(input_name,num_filter,kernel_size)
    x1 = Conv_BN(x,num_filter,kernel_size)
    return Add()([input_name,x1])

def Conv_R_A(input_name,num_filter,kernel_size):
    x = Conv_BN(input_name,num_filter,kernel_size)
    x1 = Conv_BN(x,num_filter,kernel_size)
    return Add()([input_name,x1])

In [5]:
def make_m(f_size):
    input1 = Input(shape=(f_size,1))
    conv1 = Conv_BN(input1,16,32)

    conv_r1 = Conv_Res(conv1,16,32)
    conv2  = Conv1D(32,32,strides=2,padding='same')(conv_r1)
    conv2_x = BatchNormalization()(conv2)
    conv2_y = Activation('relu')(conv2_x)
    conv_r2 = Conv_Res(conv2_y,32,32)
    conv_r3 = Conv_Res(conv_r2,32,32)

    conv3  = Conv1D(64,32,strides=2,padding='same')(conv_r3)
    conv3_x = BatchNormalization()(conv3)
    conv3_y = Activation('relu')(conv3_x)
    conv_r4 = Conv_Res(conv3_y,64,32)

    bl = Bidirectional(LSTM(units=40,return_sequences=True))(conv_r4)
    gap1 = GlobalAveragePooling1D()(bl)

    drop = Dropout(0.5)(gap1)
    
    fc1 = Dense(100, activation='relu')(drop)
    fc2 = Dense(5, activation='softmax')(fc1)
    model1 = Model(input1,fc2)
    model1.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
    return model1

In [6]:
f_size = X_train.shape[1]
class_num = 5

#============================================#

lr = 0.01
batch_size=32

Y_train = keras.utils.to_categorical(Y_train, num_classes=class_num)


model = make_m(f_size)

best_SE = 0
best_ACC = 0
best_model = make_m(f_size)
patience = 10
pcnt = 0

bin_label = lambda x: min(1,x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 201, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 201, 16)      528         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 201, 16)      64          conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 201, 16)      0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [7]:

for e in range(1, 40+1):

    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1, verbose=0)

    y_pred = model.predict(X_valid)
    y_pred = np.argmax(y_pred, axis=1)
    acc = np.sum(y_pred==Y_valid)/len(Y_valid)

    y_true = list(map(bin_label, Y_valid))
    y_pred = list(map(bin_label, y_pred))
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp/(tp+fn)
    SP = tn/(fp+tn)

    if SE+acc > best_SE+best_ACC:
        best_SE, best_ACC = SE, acc
        best_model.set_weights(model.get_weights())
        pcnt = 0
    else:
        pcnt += 1
    
    print("Epoch: %d | SE: %.4f | Best SE: %.4f | ACC: %.4f | Best ACC: %.4f | AUC: %.4f | SP: %.4f" %(e, SE, best_SE, acc, best_ACC, auc, SP))

    if pcnt==patience:
        y_pred = best_model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        acc = np.sum(y_pred==Y_test)/len(Y_test)
#         y_true = list(map(bin_label, Y_test))
#         y_pred = list(map(bin_label, y_pred))
#         auc = roc_auc_score(y_true, y_pred)
#         tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
#         SE = tp/(tp+fn)
#         SP = tn/(fp+tn)
        print(metrics.classification_report(Y_test, y_pred, target_names=['N','S','V','F','Q']))
#         print(" Test | SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f" %(SE, acc, auc, SP, best_SE, best_ACC))
#         with open("./result/"+save, "a") as fw:
#             fw.write("SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f\n" %(SE, acc, auc, SP, best_SE, best_ACC))
    

Epoch: 1 | SE: 0.7307 | Best SE: 0.7307 | ACC: 0.7934 | Best ACC: 0.7934 | AUC: 0.7779 | SP: 0.8251
Epoch: 2 | SE: 0.7088 | Best SE: 0.7088 | ACC: 0.8330 | Best ACC: 0.8330 | AUC: 0.7884 | SP: 0.8681
Epoch: 3 | SE: 0.8868 | Best SE: 0.7088 | ACC: 0.5129 | Best ACC: 0.8330 | AUC: 0.7045 | SP: 0.5221
Epoch: 4 | SE: 0.3030 | Best SE: 0.7088 | ACC: 0.6321 | Best ACC: 0.8330 | AUC: 0.5132 | SP: 0.7233
Epoch: 5 | SE: 0.7514 | Best SE: 0.7514 | ACC: 0.8191 | Best ACC: 0.8191 | AUC: 0.8035 | SP: 0.8557
Epoch: 6 | SE: 0.6740 | Best SE: 0.7514 | ACC: 0.6630 | Best ACC: 0.8191 | AUC: 0.6751 | SP: 0.6762
Epoch: 7 | SE: 0.6490 | Best SE: 0.7514 | ACC: 0.8630 | Best ACC: 0.8191 | AUC: 0.7799 | SP: 0.9109
Epoch: 8 | SE: 0.7845 | Best SE: 0.7514 | ACC: 0.6718 | Best ACC: 0.8191 | AUC: 0.7336 | SP: 0.6828
Epoch: 9 | SE: 0.8368 | Best SE: 0.8368 | ACC: 0.7491 | Best ACC: 0.7491 | AUC: 0.7981 | SP: 0.7593
Epoch: 10 | SE: 0.7557 | Best SE: 0.7557 | ACC: 0.8715 | Best ACC: 0.8715 | AUC: 0.8323 | SP: 0.9088

/home/lingang/anaconda3/envs/lzy/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           N       0.95      0.82      0.88     19629
           S       0.17      0.03      0.04       505
           V       0.35      0.94      0.51      2139
           F       0.00      0.00      0.00       392
           Q       0.00      0.00      0.00         6

    accuracy                           0.80     22671
   macro avg       0.29      0.36      0.29     22671
weighted avg       0.86      0.80      0.81     22671

Epoch: 34 | SE: 0.7358 | Best SE: 0.8091 | ACC: 0.7358 | Best ACC: 0.8485 | AUC: 0.7422 | SP: 0.7486
Epoch: 35 | SE: 0.7730 | Best SE: 0.8091 | ACC: 0.8510 | Best ACC: 0.8485 | AUC: 0.8262 | SP: 0.8795
Epoch: 36 | SE: 0.7973 | Best SE: 0.7973 | ACC: 0.8780 | Best ACC: 0.8780 | AUC: 0.8540 | SP: 0.9107
Epoch: 37 | SE: 0.5527 | Best SE: 0.7973 | ACC: 0.6591 | Best ACC: 0.8780 | AUC: 0.6245 | SP: 0.6962
Epoch: 38 | SE: 0.7709 | Best SE: 0.7973 | ACC: 0.8554 | Best ACC: 0.8780 | AUC: 0.8280 | SP: 0.8850
Epoch:

In [20]:

f_size = X_train.shape[1]
class_num = 5

#============================================#

lr = 0.01
batch_size=32

Y_train = keras.utils.to_categorical(Y_train, num_classes=class_num)

def make_model():
    model = Sequential()
    model.add(Conv1D(18, 7, activation='relu', input_shape=(f_size,1)))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(18, 7, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(class_num, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr))
    return model
model = make_model()

best_SE = 0
best_ACC = 0
best_model = make_model()
patience = 10
pcnt = 0

bin_label = lambda x: min(1,x)

for e in range(1, 40+1):

    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1, verbose=0)

    y_pred = model.predict(X_valid)
    y_pred = np.argmax(y_pred, axis=1)
    acc = np.sum(y_pred==Y_valid)/len(Y_valid)

    y_true = list(map(bin_label, Y_valid))
    y_pred = list(map(bin_label, y_pred))
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp/(tp+fn)
    SP = tn/(fp+tn)

    if SE+acc > best_SE+best_ACC:
        best_SE, best_ACC = SE, acc
        best_model.set_weights(model.get_weights())
        pcnt = 0
    else:
        pcnt += 1
    
    print("Epoch: %d | SE: %.4f | Best SE: %.4f | ACC: %.4f | Best ACC: %.4f | AUC: %.4f | SP: %.4f" %(e, SE, best_SE, acc, best_ACC, auc, SP))

    if pcnt==patience:
        y_pred = best_model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        acc = np.sum(y_pred==Y_test)/len(Y_test)
#         y_true = list(map(bin_label, Y_test))
#         y_pred = list(map(bin_label, y_pred))
#         auc = roc_auc_score(y_true, y_pred)
#         tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
#         SE = tp/(tp+fn)
#         SP = tn/(fp+tn)
        print(metrics.classification_report(Y_test, y_pred, target_names=['N','S','V','F','Q']))
#         print(" Test | SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f" %(SE, acc, auc, SP, best_SE, best_ACC))
#         with open("./result/"+save, "a") as fw:
#             fw.write("SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f\n" %(SE, acc, auc, SP, best_SE, best_ACC))
    

Epoch: 1 | SE: 0.7375 | Best SE: 0.7375 | ACC: 0.7567 | Best ACC: 0.7567 | AUC: 0.7614 | SP: 0.7853
Epoch: 2 | SE: 0.7872 | Best SE: 0.7872 | ACC: 0.7915 | Best ACC: 0.7915 | AUC: 0.8125 | SP: 0.8378
Epoch: 3 | SE: 0.3017 | Best SE: 0.7872 | ACC: 0.7085 | Best ACC: 0.7915 | AUC: 0.5504 | SP: 0.7991
Epoch: 4 | SE: 0.7530 | Best SE: 0.7872 | ACC: 0.8080 | Best ACC: 0.7915 | AUC: 0.7969 | SP: 0.8408
Epoch: 5 | SE: 0.7561 | Best SE: 0.7872 | ACC: 0.7512 | Best ACC: 0.7915 | AUC: 0.7654 | SP: 0.7746
Epoch: 6 | SE: 0.7409 | Best SE: 0.7872 | ACC: 0.8131 | Best ACC: 0.7915 | AUC: 0.7946 | SP: 0.8483
Epoch: 7 | SE: 0.8274 | Best SE: 0.8274 | ACC: 0.7946 | Best ACC: 0.7946 | AUC: 0.8164 | SP: 0.8055
Epoch: 8 | SE: 0.7000 | Best SE: 0.8274 | ACC: 0.7827 | Best ACC: 0.7946 | AUC: 0.7597 | SP: 0.8193
Epoch: 9 | SE: 0.7291 | Best SE: 0.8274 | ACC: 0.7721 | Best ACC: 0.7946 | AUC: 0.7629 | SP: 0.7968
Epoch: 10 | SE: 0.7088 | Best SE: 0.8274 | ACC: 0.7938 | Best ACC: 0.7946 | AUC: 0.7670 | SP: 0.8252

In [16]:
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(metrics.classification_report(Y_test, y_pred, target_names=['N','S','V','F','Q']))

              precision    recall  f1-score   support

           N       0.94      0.77      0.85     19629
           S       0.34      0.17      0.23       505
           V       0.29      0.84      0.43      2139
           F       0.00      0.00      0.00       392
           Q       0.00      0.00      0.00         6

    accuracy                           0.75     22671
   macro avg       0.31      0.36      0.30     22671
weighted avg       0.85      0.75      0.78     22671



In [15]:
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
acc = np.sum(y_pred==Y_test)/len(Y_test)
y_true = list(map(bin_label, Y_test))
y_pred = list(map(bin_label, y_pred))
auc = roc_auc_score(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
SE = tp/(tp+fn)
SP = tn/(fp+tn)
print(" Test | SE: %.4f | ACC: %.4f | AUC: %.4f | SP: %.4f | valid SE: %.4f | valid ACC: %.4f" %(SE, acc, auc, SP, best_SE, best_ACC))


 Test | SE: 0.9505 | ACC: 0.9848 | AUC: 0.9723 | SP: 0.9940 | valid SE: 0.9542 | valid ACC: 0.9849


In [23]:
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

In [24]:

print(metrics.classification_report(Y_test, y_pred, target_names=['N','S','V','F','Q']))

              precision    recall  f1-score   support

           N       0.99      0.99      0.99     18113
           S       0.83      0.78      0.80       517
           V       0.98      0.96      0.97      1470
           F       0.87      0.82      0.85       161
           Q       0.99      0.99      0.99      1632

    accuracy                           0.98     21893
   macro avg       0.93      0.91      0.92     21893
weighted avg       0.98      0.98      0.98     21893



In [18]:
Y_test[:5]

array([0, 3, 0, 0, 0])

In [21]:
y_pred[:5]

[0, 0, 0, 0, 0]